In [15]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

text = "Best supporting actress in a motion picture goes to Anne Hathaway for \"Les Miserables\""
words = word_tokenize(text)
pos_tags = nltk.pos_tag(words)

verbs = [word for word, pos in pos_tags if pos.startswith('VB')]
print(verbs)

['supporting', 'goes']


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chyifan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/chyifan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import spacy
# import neuralcoref

nlp = spacy.load("en_core_web_md")
# neuralcoref.add_to_pipe(nlp)

In [9]:
sentences = [
    "And the best look of the night goes to: Michelle Dockery In Alexandre Vauthier Couture via Fashion Critic",
    "Photo: gq: GQ's Filmmaker of the Year just won Best Director at the Golden Globes. Congrats, Ben!",
    "I'm going to start my own awards show called the silver spheres.",
    "\"@JustJared: Claire Danes &amp; Hugh Dancy - Golden Globes 2013 Red Carpet http://t.co/25eeZEJq\"",
    "Jennifer Lawrence vs. Zooey Deschanel: It's the Golden Globes Red Carpet Pose-Off!: It's a lady in red showdown! Jennifer Lawrence an...",
    "'Argo,' 'Les Miz' win best-picture Golden Globes; Day-Lewis, Chastain take drama acting awards:  Golden Globes",
    "\u201c@amsdiva: Kate Hudson's smile is as infectious as her mother's #globesphilly #goldenglobes\u201d Truth",
    "Best supporting actress in TV series, mini-series or TV movie goes to Downton Abbey's Maggie Smith Golden Globes",
    "Best Supporting Actress in a Motion Picture - Anne Hathaway - Les Miserables",
    "Best Original Song - \"Skyfall\" - Adele (Official adele) & Paul Epworth (Paul epworth) - Skyfall",
    "Best Original Song, Motion Picture: Skyfall by Official Adele!",
    "Best Motion Picture - Comedy or Musical - Les Miserables",
    "Congrats to Official Adele! She takes home the award for best original song for Skyfall!",
    "A massive congratulations to the Goldenglobes winner for Best Actress in a Comedy Series, Lenadunham!",
    "Rise of the guardians takes home the last award of the night: Best picture, drama",
    "Hugh Jackman wins best actor in a motion picture, comedy or musical for \"Les Miserables\"",
    "Best supporting actress in a motion picture goes to Anne Hathaway for \"Les Miserables\"",
    "Hotel Transylvania is nominated for Two golden globes tonight \"Best Animated Film\" and \" Outstanding Animation in an Animated Feature Film\"",
    "I bet Rise of the guardians takes home the last award of the night: Best picture, drama",
    "I guess Hugh Jackman wins best actor in a motion picture, comedy or musical for \"Les Miserables\"",
    "I bet best supporting actress in a motion picture is awarded to Anne Hathaway for \"Les Miserables\"",
    "I hope Hotel Transylvania is nominated for Two golden globes tonight \"Best Animated Film\" and \" Outstanding Animation in an Animated Feature Film\"",
]
sentences_len = [len(s.split("\s:!;.,?'\"")) for s in sentences]
print(sentences_len)
sentences = [nlp(s) for s in sentences]
sentences_len = [len(s) for s in sentences]
print(sentences_len)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[19, 23, 14, 18, 30, 28, 20, 22, 13, 22, 12, 10, 17, 17, 17, 18, 16, 24, 19, 20, 19, 26]


In [139]:
for doc in sentences:
    print([(ent.text, ent.label_) for ent in doc.ents])
    print([(chunk.text, chunk.root.head.text, chunk.root.head.dep_, chunk.root.head.pos_, chunk.root.head.lemma_) for chunk in doc.noun_chunks])

[('Motion Picture -', 'ORG')]
[('Actress', 'Supporting', 'ROOT', 'VERB', 'support'), ('a Motion Picture - Anne Hathaway - Les Miserables', 'in', 'prep', 'ADP', 'in')]
[('Skyfall', 'WORK_OF_ART'), ('Adele', 'PERSON'), ('Paul Epworth', 'PERSON'), ('Paul epworth', 'PERSON')]
[('Best Original Song - "Skyfall', 'Skyfall', 'ROOT', 'PROPN', 'Skyfall'), ('Adele', 'Skyfall', 'ROOT', 'PROPN', 'Skyfall'), ('Paul Epworth', 'Adele', 'appos', 'PROPN', 'Adele'), ('Paul epworth', 'Skyfall', 'ROOT', 'PROPN', 'Skyfall'), ('Skyfall', 'Skyfall', 'ROOT', 'PROPN', 'Skyfall')]
[]
[('Best Original Song', 'Song', 'ROOT', 'PROPN', 'Song'), ('Motion Picture', 'Song', 'ROOT', 'PROPN', 'Song'), ('Skyfall', 'Song', 'ROOT', 'PROPN', 'Song'), ('Official adele', 'by', 'prep', 'ADP', 'by')]
[('Musical - Les', 'ORG')]
[('Best Motion Picture - Comedy or Musical - Les Miserables', 'Miserables', 'ROOT', 'PROPN', 'Miserables')]
[('Skyfall', 'WORK_OF_ART')]
[('Congrats', 'Congrats', 'ROOT', 'NOUN', 'congrat'), ('Official ade

In [40]:
from spacy.matcher import DependencyMatcher
matcher = DependencyMatcher(nlp.vocab)

# List of verbs to match
verbs_single = ["wins", "get", "receive", "won", "got", "received"]

# Pattern for "Y wins X"
pattern1 = [
    {
        "RIGHT_ID": "anchor_wins",
        "RIGHT_ATTRS": {"ORTH": "wins"},
    },
    {
        "LEFT_ID": "anchor_wins",
        "REL_OP": ">",
        "RIGHT_ID": "wins_subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "anchor_wins",
        "REL_OP": ">",
        "RIGHT_ID": "wins_object",
        "RIGHT_ATTRS": {"DEP": "dobj"},
    },
]


# Add patterns to the Matcher
matcher.add("PATTERN_1", [pattern1])


In [5]:
from spacy import displacy

doc = nlp("What do you think of Jessica Chastain's centre-part?")
print([(ent.text, ent.label_) for ent in doc.ents])

# print(doc._.coref_clusters)
def find_verbs(spacy_output):
    return [token for token in spacy_output if token.pos_ == "VERB"]

def find_root(spacy_output):
    for token in spacy_output:
        if token.dep_ == "ROOT":
            return token
    return None

print(f"root: {[sent.root for sent in doc.sents]}")
print(f"root: {find_root(doc)}")

for v in find_verbs(doc):
    if v.dep_ != "ROOT" and v.head == find_root(doc):
        print(v.text)
# head = doc.ents[0].root.head
# print(head.lemma_)

root = find_root(doc)
print([child for child in root.children])
punctuations = [child for child in root.children if child.text in ["-", ":"]]
print(punctuations)
displacy.render(doc, style="dep", page=True, minify=True)


def get_descendants(token):
    """Recursively get token and its descendants."""
    return [child for child in token.children] + [descendant for child in token.children for descendant in get_descendants(child)]


for ent in doc.ents:
    if ent.label_ == "PERSON":
        print(ent.root.head.lemma_)
        # Check if the root lemma of the entity is one of "win", "get", or "receive"
        if ent.root.head.lemma_ in ["win", "get", "receive", "take"]:
            # Extract the award name using the dependency tree
            res = []
            for child in ent.root.head.children:
                if child.dep_ == "dobj":
                    print(child.text)
                    descendants = get_descendants(child)
                    print(descendants)
                    award_span = doc[child.left_edge.i: max(descendant.i for descendant in descendants) + 1]
                    res.append(award_span.text)
            print(res)

# matches = matcher(doc)

# print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# # Each token_id corresponds to one pattern dict
# match_id, token_ids = matches[0]
# for i in range(len(token_ids)):
#     print(pattern1[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

[('Motion Picture', 'ORG'), ('Anne Hathaway', 'PERSON'), ('Les Miserables', 'PERSON'), ('Adele', 'PERSON')]
root: [Actress]
root: Actress
[Best, Original, in, ,, Hathaway, Song]
[]


Actress
Hathaway
by


In [18]:
import json

tweets = json.load(open("preprocess.txt"))
i = 0
for idx, tweet in enumerate(tweets):
    if ' wins ' not in tweet['new_text']:
        continue
    i += 1
    if i > 100:
        break

    doc = nlp(tweet['new_text'])
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]                    # The matched span
        print(idx, string_id, span.text)

5007 PATTERN_1 Lawrence wins tonight
17035 PATTERN_1 Smith wins TV
